# EDA completo – vehicles_us.csv
Este cuaderno reúne: carga robusta, diagnóstico, limpieza mínima con criterios, *features* y gráficas base.


In [12]:
import os
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Forzar renderer solo fuera de Streamlit
if os.environ.get("STREAMLIT_RUNTIME") is None:
    try:
        pio.renderers.default = "vscode"  # alternativa: "notebook_connected"
    except Exception:
        pio.renderers.default = "browser"

pd.set_option("display.max_columns", 100)  # solo vista
print("Librerías cargadas. Plotly renderer:", pio.renderers.default)


Librerías cargadas. Plotly renderer: vscode


In [13]:
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "statsmodels"])

0

## Cargar datos


In [ ]:
CSV_PATH = Path("../data/vehicles_us.csv")  # cambia si usas otro archivo

assert CSV_PATH.exists(), f"No se encontró el archivo: {CSV_PATH.resolve()}"

df = pd.read_csv(
    CSV_PATH,
    low_memory=False,
    # encoding="utf-8",   # si ves UnicodeDecodeError, prueba "latin-1" o "utf-8-sig"
    # na_values=["NA","NaN",""],
    # on_bad_lines="skip",  # usarlo solo si hay filas corruptas
)

df_raw = df.copy()

print(f"✅ Cargado: {CSV_PATH.name}")
print(f"Filas x Columnas: {df.shape[0]:,} x {df.shape[1]}")
print("Columnas:", list(df.columns))
display(df.head())


✅ Cargado: vehicles_us.csv
Filas x Columnas: 51,525 x 13
Columnas: ['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'transmission', 'type', 'paint_color', 'is_4wd', 'date_posted', 'days_listed']


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


## Diagnóstico express


In [15]:
print("Tipos de datos:")
display(df.dtypes)

print("\n% de nulos por columna (Top 10):")
display((df.isna().mean().sort_values(ascending=False) * 100).head(10))

print("\nDuplicados totales:", df.duplicated().sum())


Tipos de datos:


price             int64
model_year      float64
model            object
condition        object
cylinders       float64
fuel             object
odometer        float64
transmission     object
type             object
paint_color      object
is_4wd          float64
date_posted      object
days_listed       int64
dtype: object


% de nulos por columna (Top 10):


is_4wd         50.369723
paint_color    17.985444
odometer       15.316836
cylinders      10.208637
model_year      7.023775
condition       0.000000
model           0.000000
price           0.000000
fuel            0.000000
type            0.000000
dtype: float64


Duplicados totales: 0


## Conversión de fecha (si existe)


In [16]:
if "date_posted" in df.columns:
    df["date_posted"] = pd.to_datetime(df["date_posted"], errors="coerce")
    print("✓ 'date_posted' convertido a datetime (coerce).")
else:
    print("No existe columna 'date_posted'")


✓ 'date_posted' convertido a datetime (coerce).


## Limpieza mínima + features (criterios documentados)


In [17]:
df = df.copy()
n0 = len(df)

# Tipos discretos como enteros con nulos
for col in ["model_year", "cylinders", "is_4wd"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").round().astype("Int64")

# Numéricos clave; odometer entero
for col in ["price", "odometer", "days_listed"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
if "odometer" in df.columns:
    df["odometer"] = df["odometer"].round().astype("Int64")

# Reglas de dominio (valores imposibles)
if "price" in df.columns:
    df = df[df["price"] > 0]
if "odometer" in df.columns:
    df = df[df["odometer"] >= 0]
if "days_listed" in df.columns:
    df = df[df["days_listed"] >= 0]

# Categóricas: normalizar y nulos -> 'unknown' (para EDA)
for col in ["condition","fuel","transmission","type","paint_color","model"]:
    if col in df.columns and df[col].dtype == "object":
        df[col] = df[col].fillna("unknown").str.strip().str.lower()

# Features
if "date_posted" in df.columns:
    df["posted_year"] = df["date_posted"].dt.year
    df["posted_month"] = df["date_posted"].dt.to_period("M").dt.to_timestamp()
if {"posted_year","model_year"} <= set(df.columns):
    df["age"] = (df["posted_year"] - df["model_year"]).astype("Int64")
if "model" in df.columns:
    df["brand"] = df["model"].str.split().str[0]

n1 = len(df)
print(f"Filas removidas por reglas de dominio: {n0 - n1}")
display(df.head())
display(df.dtypes)


Filas removidas por reglas de dominio: 7892


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,posted_year,posted_month,age,brand
0,9400,2011,bmw x5,good,6,gas,145000,automatic,suv,unknown,1,2018-06-23,19,2018,2018-06-01,7,bmw
1,25500,<NA>,ford f-150,good,6,gas,88705,automatic,pickup,white,1,2018-10-19,50,2018,2018-10-01,<NA>,ford
2,5500,2013,hyundai sonata,like new,4,gas,110000,automatic,sedan,red,<NA>,2019-02-07,79,2019,2019-02-01,6,hyundai
4,14900,2017,chrysler 200,excellent,4,gas,80903,automatic,sedan,black,<NA>,2019-04-02,28,2019,2019-04-01,2,chrysler
5,14990,2014,chrysler 300,excellent,6,gas,57954,automatic,sedan,black,1,2018-06-20,15,2018,2018-06-01,4,chrysler


price                    int64
model_year               Int64
model                   object
condition               object
cylinders                Int64
fuel                    object
odometer                 Int64
transmission            object
type                    object
paint_color             object
is_4wd                   Int64
date_posted     datetime64[ns]
days_listed              int64
posted_year              int32
posted_month    datetime64[ns]
age                      Int64
brand                   object
dtype: object

## Capa de visualización sin outliers (df intacto para análisis)


In [18]:
df_vis = df.copy()
if "price" in df_vis.columns:
    q1, q99 = df_vis["price"].quantile([0.01, 0.99])
    df_vis = df_vis.query("@q1 <= price <= @q99")
print("df_vis shape:", df_vis.shape)


df_vis shape: (43199, 17)


## Visualizaciones clave


In [19]:
# Histograma de precios


if {"price"} <= set(df_vis.columns):
    px.histogram(df_vis, x="price", nbins=60, title="Distribución de precios").show()

# Dispersión odometer vs price (color por año modelo si existe)
if {"odometer","price"} <= set(df_vis.columns):
    color_col = "model_year" if "model_year" in df_vis.columns else None
    px.scatter(df_vis.dropna(subset=["odometer","price"]),
               x="odometer", y="price", color=color_col,
               trendline="lowess", title="Precio vs Odómetro").show()

# Boxplot de precio por condición
if {"condition","price"} <= set(df_vis.columns):
    px.box(df_vis, x="condition", y="price", title="Precio por condición").show()

# Barras: conteo por tipo de vehículo
if {"type"} <= set(df_vis.columns):
    counts = df_vis["type"].value_counts().reset_index()
    counts.columns = ["type","count"]
    px.bar(counts, x="type", y="count", title="Conteo por tipo").update_layout(xaxis_tickangle=-45).show()

# Serie temporal: publicaciones por mes
if {"posted_month"} <= set(df_vis.columns):
    ts = df_vis.groupby("posted_month").size().rename("count").reset_index()
    px.line(ts, x="posted_month", y="count", title="Publicaciones por mes").show()

# Precio vs edad del vehículo
if {"age","price"} <= set(df_vis.columns):
    px.scatter(df_vis.dropna(subset=["age","price"]),
               x="age", y="price", trendline="lowess",
               title="Precio vs Edad del vehículo").show()
